In [18]:
import pandas as pd
import numpy as np
import requests
import os

# Calculate Democratic Control

In [19]:
url = 'https://en.wikipedia.org/wiki/Political_party_strength_in_U.S._states'
r = requests.get(url)
dfs = pd.read_html(r.text)
df_pol = dfs[3]
df_pol = df_pol[['State', 'Governor', 'State Senate', 'State House']]
df_pol.head()
# manually removed parenthesis and corrected Nebraska

,State,Governor,State Senate,State House
0,Alabama,Republican,Republican 27-8,Republican 77-28
1,Alaska,Republican,Republican 13-7,Coalition 25-15(a)
2,Arizona,Republican,Republican 17-13,Republican 31-29
3,Arkansas,Republican,Republican 26-9,Republican 76-24
4,California,Democratic,Democratic 29-11,Democratic 60-20


In [20]:
# def dems_percent(c):
#     l = c.strip().split(' ')
#     nums = l[1].split('-')
#     nums = [int(n) for n in nums]
#     if l[0] == 'Republican':
#         dem = nums[1]
#         return round(dem/sum(nums)*100, 2)
#     elif l[0] == 'Democratic':
#         dem = nums[0]
#         return round(dem/sum(nums)*100, 2)
#     elif l[0] == 'Coalition':
#         # alaska is complicated
#         return 37.5
# df_pol = pd.read_csv('state_pol_data.csv')
# df_pol['dems_perc_senate'] = df_pol['State Senate'].apply(dems_percent)
# df_pol['dems_perc_house'] = df_pol['State House'].apply(dems_percent)
# df_pol['dem_gov'] = df_pol['Governor'] == 'Democratic'
# df_pol = df_pol.drop(['Governor', 'State Senate', 'State House'], axis=1)
# df_pol.to_csv('state_pol_data.csv', index=False)
df_pol = pd.read_csv('state_pol_data.csv')
df_pol.head()

,State,dems_perc_senate,dems_perc_house,dem_gov
0,Alabama,22.86,26.67,False
1,Alaska,35.00,37.50,False
2,Arizona,43.33,48.33,False
3,Arkansas,25.71,24.00,False
4,California,72.50,75.00,True


# Combine data streams

In [21]:
# create giant df
df = pd.read_csv('state_abbr.csv', header=None)
df.columns = ['state', 'abbr']
df.index = df['state']
df.drop(['state'], axis=1, inplace=True)

# combine housing
housing_df = pd.read_csv('housing_lgbt.csv')
df['housing_so'] = 0
df['housing_gi'] = 0
for i, r in housing_df.iterrows():
    s = r['state']
    df.at[s, 'housing_so'] = r['so']
    df.at[s, 'housing_gi'] = r['gi']
    
# combine employment
employment_df = pd.read_csv('employment_lgbt.csv')
df['employment_public_so'] = 0
df['employment_public_gi'] = 0
df['employment_private_so'] = 0
df['employment_private_gi'] = 0
for i, r in employment_df.iterrows():
    s = r['state']
    df.at[s, 'employment_public_so'] = r['public_so']
    df.at[s, 'employment_public_gi'] = r['public_gi']
    df.at[s, 'employment_private_so'] = r['private_so']
    df.at[s, 'employment_private_gi'] = r['private_si']

# combine conversion
df['conversion_ban'] = 0
df_conv = pd.read_csv('conversion.csv')
for i, r in df_conv.iterrows():
    s = r['state']
    df.at[s, 'conversion_ban'] = r['conversion_ban']    

# combine approval
approval_df = pd.read_csv('prri_discrimination_2018.csv')
df['poll_favor'] = np.nan
for i, r in approval_df.iterrows():
    s = r['state']
    df.at[s, 'poll_favor'] = r['favor']

# sorry dc :(
df = df[df['abbr'] != 'DC']
    
# combine democratic control
df_pol = pd.read_csv('state_pol_data.csv')
print(df_pol.head())
df[['dems_perc_senate', 'dems_perc_house', 'dem_gov']] = df_pol[['dems_perc_senate', 'dems_perc_house', 'dem_gov']]
df['dems_perc_senate'] = list(df_pol['dems_perc_senate'])
df['dems_perc_house'] = list(df_pol['dems_perc_house'])
df['dem_gov'] = list(df_pol['dem_gov'])

df

        State  dems_perc_senate  dems_perc_house  dem_gov
0     Alabama             22.86            26.67    False
1      Alaska             35.00            37.50    False
2     Arizona             43.33            48.33    False
3    Arkansas             25.71            24.00    False
4  California             72.50            75.00     True


,abbr,housing_so,housing_gi,employment_public_so,employment_public_gi,employment_private_so,employment_private_gi,conversion_ban,poll_favor,dems_perc_senate,dems_perc_house,dem_gov
state,,,,,,,,,,,,
Alabama,AL,0,0,0,0,0,0,0,59.0,22.86,26.67,False
Alaska,AK,0,0,1,0,0,0,0,68.0,35.00,37.50,False
Arizona,AZ,0,0,1,0,0,0,0,68.0,43.33,48.33,False
Arkansas,AR,0,0,0,0,0,0,0,56.0,25.71,24.00,False
California,CA,1,1,1,1,1,1,1,73.0,72.50,75.00,True
Colorado,CO,1,1,1,1,1,1,0,72.0,54.29,63.08,True
Connecticut,CT,1,1,1,1,1,1,1,76.0,63.89,60.93,True
Delaware,DE,1,1,1,1,1,1,1,71.0,57.14,63.41,True
Florida,FL,0,0,0,0,0,0,0,68.0,42.50,39.17,False


In [22]:
df.to_csv('all_data.csv')